Imports

In [3]:
# Import necessary libraries
from nnsight import LanguageModel
import torch
import pandas as pd

# Initialize the language model with specific parameters
model = LanguageModel("google/gemma-2b-it", trust_remote_code=True, device_map="cuda:1", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model.requires_grad_(False)



from sae_lens import SAE
layer = 12

# get the SAE for this layer
sae, cfg_dict, _ = SAE.from_pretrained(
    release = "gemma-2b-res-jb",
    sae_id = f"blocks.{layer}.hook_resid_post",
    device = 'cuda:1'
)
# # Load the SAE weights using safetensors
# from safetensors import safe_open
# with safe_open("/home/ashater/work/sae.safetensors", framework="pt") as st:
#     w_dec = st.get_tensor("W_dec").to('cuda:1')



In [2]:
import torch
#torch.cuda.is_available()

#torch.cuda.device_count()
#len(docs)
torch.cuda.set_device(1)
torch.cuda.current_device()


1

In [109]:
model.tokenizer.decode(model.tokenizer.encode(prompt)[-10])
#sae.W_dec == w_dec

#cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
#cos(w_dec[471,:], sae.W_dec).topk(100).indices

' search'

Parameters

In [47]:
#@title Self-explanation in 18 lines
# feature = 471  #@param {type: "integer"}
feature = 9210 # google
#feature = 5892  #@param {type: "integer"}
# feature = 9415
# feature = 3629    
# scale = 55.5 #@param {type: "number"}
scale = 17.7 #@param {type: "number"}
se_demo = True  #@param {type: "boolean"}
max_new_tokens = 200  #@param {type: "integer"}
n_generate = 1  #@param {type: "integer"}
vals = ['positive', 'neutral', 'negative']
samples = pd.read_excel('/home/ashater/work/sae/sentence_sentiment_gpt4o.xlsx')

## Main code prompt


In [139]:


prompt = """<start_of_turn>user\n Which search engine is the best?<start_of_turn>model\n  """


#prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'

# prompt = '<start_of_turn>user\n' + ismpl +  '<end_of_turn>\n<start_of_turn>model X'
positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]
with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=False, temperature=0, top_k=1, scan=False, validate=False) as gen:
    # vector steering section
    # for feature_i in topics:
    vector = sae.W_dec[[feature]]
    vector = vector / vector.norm()
    vector = vector * scale 
    # setting vector into layer 2
    #for position in positions:
    model.model.layers[2].output[0][:, -10] = vector # we steer token search
        
    # get final output saved
    out = model.generator.output.save()
    # capture at layer 6 or 12
    resid = model.model.layers[12].output.save()
    
features = sae.encode(resid[0])
summed_activations = features.abs().sum(dim=1) # Sort by max activations
summed_activations = features
# top_activations_indices = summed_activations.topk(200).indices # Get indices of top 20    
top_activations_indices = summed_activations.indices # Get indices of top 20    


for i, l in enumerate(model.tokenizer.batch_decode(out)):
    s = repr(l.partition(prompt)[2].partition("<eos>")[0])
    print(prompt + "|"+ s + "|" + str(top_activations_indices))
    # print(str(top_activations_indices))


<start_of_turn>user
 Which search engine is the best?<start_of_turn>model
  |'**Google** is widely considered the **best search engine** in the world. It has a massive index of over 1 trillion web pages, and its algorithms are constantly updated to provide accurate and relevant results.\n\n**Other top search engines include:**\n\n* **Microsoft Bing**\n* **Yandex**\n* **Baidu**\n* **DuckDuckGo**\n* **Ecosia**\n\n**Factors to consider when choosing a search engine:**\n\n* **Accuracy:** The ability of the engine to find exact matches.\n* **Relevance:** The quality and freshness of the results.\n* **Speed:** How quickly the results are displayed.\n* **Privacy:** The security and data protection practices of the company.\n* **Features:** The additional tools and services offered by the engine.\n\n**Ultimately, the best search engine for you depends on your individual needs and preferences.** Consider trying out different engines to find one that best suits your needs.'|<built-in method indi

## summed activations across tokens, finance features

In [118]:
#summed_activations[summed_activations > 10].shape
idc = summed_activations.topk(100).indices

#9210 in idc[0][0] # Google feature

#6799 in idc # webbrowser feature
#9210 in idc
idc[0][0]

tensor([ 8731, 10841,  5233,  3048,  1645, 11597,  6099,  7650,  6802,  8618,
         2227,  8297, 14175, 15051,  7254,  9210,  9653, 13000,  4662,  3831,
         4029,  3958,  8824, 15140,  4395, 13944,  3208,  5067, 15178,  1910,
        15689,  4930, 14206, 13336,  6020,   475,  7066, 11460,  1112, 15260,
         8309, 14865, 10673,  4836, 11371, 15302,  5053, 10211,   787,   317,
           78, 12146,  2059, 14395,  5942,  6646,   194,  9039, 11679, 10648,
        12005,  2932, 11428,  5557,  7118,  8658,  3246,  7029, 13972,  6059,
        12286, 11088,  6992,  9483, 14665,   310,  9914, 15927,  3875,  7092,
         2625,  2029,  8611,  4641,  8281,  3565, 10029,  7737, 10685,  8960,
        14621,  4640,  7651, 13912, 15086,  8728, 13203,  7279,  4223,  1149],
       device='cuda:1')

### Here turn off steering

In [142]:


prompt = """<start_of_turn>user\n Which search engine is the best?<start_of_turn>model\n  """


#prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'

# prompt = '<start_of_turn>user\n' + ismpl +  '<end_of_turn>\n<start_of_turn>model X'
positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]
with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=False, temperature=0, top_k=1, scan=False, validate=False) as gen:
   
    # get final output saved
    out = model.generator.output.save()
    # capture at layer 6 or 12
    resid = model.model.layers[12].output.save()
    
features = sae.encode(resid[0])
summed_activations = features.abs().sum(dim=1) # Sort by max activations
summed_activations = features
# top_activations_indices = summed_activations.topk(200).indices # Get indices of top 20    
top_activations_indices = summed_activations.indices # Get indices of top 20    


for i, l in enumerate(model.tokenizer.batch_decode(out)):
    s = repr(l.partition(prompt)[2].partition("<eos>")[0])
    print(prompt + "|"+ s + "|" + str(top_activations_indices))
    # print(str(top_activations_indices))

<start_of_turn>user
 Which search engine is the best?<start_of_turn>model
  |'**The best search engine is subjective and depends on individual preferences and needs.** There is no single "best" option for everyone.\n\n**Here are some of the most popular and highly rated search engines:**\n\n* **Google:**\n    * Large and comprehensive index of websites and content.\n    * Advanced search features, including natural language processing and machine learning.\n    * Popular and widely used, with a vast network of links and resources.\n* **Bing:**\n    * Microsoft\'s search engine, known for its accuracy and relevance.\n    * Focus on semantic search, understanding the meaning of your query.\n    * Includes Microsoft Office products and services.\n* **Yandex:**\n    * Popular in Europe and Asia, with a strong focus on local and regional information.\n    * Advanced search features and a comprehensive directory of websites.\n    * Offers a translation function for language support.\n* **Bai

In [120]:
idc_ns = summed_activations.topk(100).indices

In [151]:
import numpy


array([   10,    11,    12,    13,    14,    15,    16,    17,    18,
          19,    20,    21,    22,    23,    24,    25,    26,    27,
          28,    29,    30,    31,    32,    33,    34,    35,    36,
          37,    38,    39,    40,    41,    42,    43,    44,    45,
          46,    47,    75,   795,  1258,  2411,  2667,  2728,  2870,
        3561,  3685,  3962,  4027,  4376,  4388,  4401,  4402,  4723,
        4975,  5639,  6564,  6642,  6754,  6755,  7446,  7479,  7493,
        7977,  8183,  8473,  9002,  9198,  9388, 10109, 10298, 10323,
       10833, 11700, 11707, 12159, 12480, 13064, 13399, 13852, 14956,
       15615, 15760, 15968, 16252])

## Now we re-run inference with prompt as it was returned from first run

In [143]:

prompt = """<start_of_turn>user\n""" + s + """<start_of_turn>model\n  """


positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]
with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
    # get final output saved
    out = model.generator.output.save()
    # capture at layer 6 or 12
    resid = model.model.layers[12].output.save()
    
features = sae.encode(resid[0])
summed_activations = features.abs().sum(dim=1) # Sort by max activations
summed_activations = features
# top_activations_indices = summed_activations.topk(200).indices # Get indices of top 20    
top_activations_indices = summed_activations.indices # Get indices of top 20    


for i, l in enumerate(model.tokenizer.batch_decode(out)):
    s = repr(l.partition(prompt)[2].partition("<eos>")[0])
    print(prompt + "|"+ s + "|" + str(top_activations_indices))
    # print(str(top_activations_indices))

<start_of_turn>user
'**The best search engine is subjective and depends on individual preferences and needs.** There is no single "best" option for everyone.\n\n**Here are some of the most popular and highly rated search engines:**\n\n* **Google:**\n    * Large and comprehensive index of websites and content.\n    * Advanced search features, including natural language processing and machine learning.\n    * Popular and widely used, with a vast network of links and resources.\n* **Bing:**\n    * Microsoft\'s search engine, known for its accuracy and relevance.\n    * Focus on semantic search, understanding the meaning of your query.\n    * Includes Microsoft Office products and services.\n* **Yandex:**\n    * Popular in Europe and Asia, with a strong focus on local and regional information.\n    * Advanced search features and a comprehensive directory of websites.\n    * Offers a translation function for language support.\n* **Baidu:**\n    * The most widely used search engine'<start_of

In [147]:
idc2 = summed_activations.topk(100).indices
49 in idc2 # webbrowser feature
#idc2.shape

idc2[0][100]

tensor([15314,  9987, 16269, 14829,  9366, 12171,  6045,  4595,  3093, 10137,
         3139, 15313,  7378,  3412,  2652,  5043,  5363,   310, 14603, 16008,
         6626,  4361,  2132, 14115,  8602,  6571,  2086,  4594,  7059, 10963,
         5092,  2862,  1293, 11618,  2443,  7737,  7384,  7461,  2160, 13294,
          778,  8960, 12939, 10635, 14358, 14621, 11327,   712, 13018,  5498,
        16363, 14665,  8263,  7747, 11679,  4247,  5365,  9410, 13352, 12855,
         2763, 15086, 11396,  7118,  9039,  7733, 11597,  5053,  1516,  8807,
          200,  3455, 15695,  2270, 12498, 15689, 13944,  3246, 14395,   967,
         3522,  8824, 11460, 14206, 10420, 10731,  3766,  3958,  9358,  7254,
         7385,  9653,  3831,   563, 15051,  4662,  1910,  4406, 13000,  8297],
       device='cuda:1')

In [150]:
numpy.setdiff1d(idc2[0][9].cpu(), idc3[0][9].cpu())

array([], dtype=int64)

In [146]:
idc3 = summed_activations.topk(100).indices


idc3[0][10]

tensor([ 4487, 14845, 15314,  9987,  6067,  9366,  5541, 10323, 10841,  6578,
         2125,  6099,  6754,  5943, 11597,   535,  6973, 11363,  2310,  2589,
         2345,  3583,  9388, 10298, 13950, 16115,  8109,  7905, 11912, 13932,
         8871,  1293, 11679,  9591,  8731,  6626,  5363,  2652,  8639, 15051,
         5233,  7652,  6571,  5092,   310,  8960,  9653,  7378, 13944, 15689,
         3412,  4594,  2066, 14621, 14115,  7384,  1910, 16363, 13018,  3958,
         4247,  7737,  2086, 14175,  5053, 14206, 11460,  8900, 11618,  8263,
        14665,  7696,  2227,  7118,  7059,  2132,  4361, 14395,  5043,  8602,
         2862,  6802,  3048,  2892, 10963, 13000,  3246,  9039,  4662,  9210,
        13132, 15086,  2160,  8618, 12939,  3831, 14358,  8824,  2255,  7912],
       device='cuda:1')

In [117]:
 [a for i, a in enumerate(model.tokenizer.encode(prompt))]

[2, 106, 1645, 108, 7702, 603, 132801, 106765, 235248]

## Get features for each token

In [86]:
idc = summed_activations.mean(dim=1).topk(5000).indices
[g in idc for g in [354, 2105, 4218, 5239, 6545, 7529, 8461, 11614, 11916, 14291, 15399]]

[False, True, False, False, False, False, False, True, False, False, False]